In [1]:
import tensorflow as tf
import tf_encrypted as tfe

from tensorflow.keras.losses import MeanSquaredError as tf_MeanSquaredError
from tensorflow.keras.optimizers import SGD as tf_SGD

from tf_encrypted.keras.losses import Loss
from tf_encrypted.keras import backend as KE
from tf_encrypted.keras.optimizers import SGD

import pandas as pd
import sys
sys.path.append('../')




/anaconda3/envs/py35_int64/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import numpy as np
import matplotlib.pyplot as plt

# generate random data-set
np.random.seed(0)
x = np.random.rand(2000, 1)
y = 1 + 4 * x + np.random.rand(2000, 1)

In [3]:
X_train, y_train = x[:1500], y[:1500]
X_valid, y_valid = x[1500:], y[1500:]

In [4]:
num_features = 1
training_set_size = 1500
test_set_size = 500
batch_size = 100
steps_per_epoch = (training_set_size // batch_size)
epochs = 20

# Plaintext Training

In [5]:
tf_model = tf.keras.Sequential()
tf_model.add(tf.keras.layers.Dense(1, input_shape=[num_features]))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [6]:
tf_model.compile(optimizer=tf_SGD(lr=0.01),
                 loss=tf_MeanSquaredError())

tf_model.fit(X_train,
             y_train,
             epochs=20,
             steps_per_epoch=steps_per_epoch)


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Epoch 1/20
15/15 [==============================] - 0s 7ms/step - loss: 7.4582
Epoch 2/20
15/15 [==============================] - 0s 707us/step - loss: 3.6236
Epoch 3/20
15/15 [==============================] - 0s 755us/step - loss: 1.8455
Epoch 4/20
15/15 [==============================] - 0s 829us/step - loss: 1.0184
Epoch 5/20
15/15 [==============================] - 0s 932us/step - loss: 0.6312
Epoch 6/20
15/15 [==============================] - 0s 1ms/step - loss: 0.4475
Epoch 7/20
15/15 [==============================] - 0s 1ms/step - loss: 0.3581
Epoch 8/20
15/15 [==============================] - 0s 986us/step - loss: 0.3125
Epoch 9/20
15/15 [==============================] - 0s 1ms/step - loss: 0.2873
Epoch 10/20
15/15 [==============================] - 0s 899us/step - loss: 0.2717
Epoch 11/20
15/15 [==============================] - 0s 1ms/step - loss: 0.2607
Epoch 12/20
15/15 [==============================] - 0s 999us/step - loss: 0.2519
Epoch 13/20
15/15 [================

# Private Training

In [7]:
class DataOwner:
  """Contains code meant to be executed by a data owner Player."""
  def __init__(
      self,
      player_name,
      num_features,
      training_set_size,
      test_set_size,
      batch_size
  ):
    self.player_name = player_name
    self.num_features = num_features
    self.training_set_size = training_set_size
    self.test_set_size = test_set_size
    self.batch_size = batch_size
    self.train_initializer = None
    self.test_initializer = None

  @property
  def initializer(self):
    return tf.group(self.train_initializer, self.test_initializer)

  @tfe.local_computation
  def provide_training_data(self):
    """Preprocess training dataset

    Return single batch of training dataset
    """
    def norm(x, y):
      return tf.cast(x, tf.float32), tf.expand_dims(y, 0)

    train_set = tf.data.Dataset.from_tensor_slices((X_train, y_train)) \
        .map(norm) \
        .repeat() \
        .shuffle(buffer_size=self.batch_size) \
        .batch(self.batch_size)

    train_set_iterator = train_set.make_initializable_iterator()
    self.train_initializer = train_set_iterator.initializer

    x, y = train_set_iterator.get_next()
    x = tf.reshape(x, [self.batch_size, self.num_features])
    y = tf.reshape(y, [self.batch_size, 1])

    return x, y

  @tfe.local_computation
  def provide_testing_data(self):
    """Preprocess testing dataset

    Return single batch of testing dataset
    """
    def norm(x, y):
      return tf.cast(x, tf.float32), tf.expand_dims(y, 0)

    test_set = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)) \
        .map(norm) \
        .batch(self.test_set_size)

    test_set_iterator = test_set.make_initializable_iterator()
    self.test_initializer = test_set_iterator.initializer

    x, y = test_set_iterator.get_next()
    x = tf.reshape(x, [self.test_set_size, self.num_features])
    y = tf.reshape(y, [self.test_set_size, 1])

    return x, y


In [8]:
data_owner = DataOwner('data-owner',
                       num_features,
                       training_set_size,
                       test_set_size,
                       batch_size)

x_train_pond, y_train_pond = data_owner.provide_training_data()
x_valid_pond, y_valid_pond = data_owner.provide_testing_data()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [9]:
class MSE(Loss):
  """Computes the MSE loss between true
  labels and predicted labels.
  """
  def __init__(self):
    super(MSE, self).__init__(
        mse)

  def grad(self, y_true, y_pred):
    batch_size = y_true.shape.as_list()[0] 
    batch_size_inv = 1 / batch_size  
    return 2 * (y_pred - y_true) * batch_size_inv 

def mse(y_true, y_pred):
    batch_size = y_true.shape.as_list()[0] 
    batch_size_inv = 1 / batch_size  
    out = y_true - y_pred #Internally converts y_true to PondPrivateTensor
    out=out.square() 
    mse_loss = out.reduce_sum(axis=0) * batch_size_inv # mean squared error
    return  mse_loss

In [10]:
model = tfe.keras.Sequential()
model.add(tfe.keras.layers.Dense(1, batch_input_shape=[batch_size, num_features]))

In [11]:
sess = KE.get_session()
sess.run([data_owner.initializer])

model.compile(optimizer=SGD(lr=0.01),
              loss=MSE())
# Train
model.fit(x_train_pond,
          y_train_pond,
          epochs=20,
          steps_per_epoch=steps_per_epoch)

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2', 'data-owner']


Epoch 1/20
1500/1500 [==============================] - 1s 490us/step - loss: 9.1494
Epoch 2/20
1500/1500 [==============================] - 0s 75us/step - loss: 4.4604
Epoch 3/20
1500/1500 [==============================] - 0s 76us/step - loss: 2.4138
Epoch 4/20
1500/1500 [==============================] - 0s 76us/step - loss: 1.3905
Epoch 5/20
1500/1500 [==============================] - 0s 76us/step - loss: 0.9516
Epoch 6/20
1500/1500 [==============================] - 0s 74us/step - loss: 0.6452
Epoch 7/20
1500/1500 [==============================] - 0s 74us/step - loss: 0.5274
Epoch 8/20
1500/1500 [==============================] - 0s 76us/step - loss: 0.4778
Epoch 9/20
1500/1500 [==============================] - 0s 87us/step - loss: 0.4231
Epoch 10/20
1500/1500 [==============================] - 0s 82us/step - loss: 0.4134
Epoch 11/20
1500/1500 [==============================] - 0s 77us/step - loss: 0.4035
Epoch 12/20
1500/1500 [==============================] - 0s 75us/step - l